In [0]:
import tensorflow as tf
tf.__version__ # 2.0.0

'2.0.0'

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
import os
import tensorflow as tf
import numpy as np
 
SEED = 777
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()


## recommended gpu settings by Lattari ##

# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [0]:
# Batch size
bs =  8

# img shape

img_h = 320
img_w = 320

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10, 
                                        width_shift_range=10,
                                        validation_split=0.2,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [0]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join('./', 'Classification_Dataset')

num_classes=20


In [0]:
classes = ['owl',
'galaxy' ,
'lightning' ,
'wine-bottle' ,
't-shirt' ,
'waterfall' ,
'sword' ,
'school-bus' ,
'calculator' ,
'sheet-music' ,
'airplanes' ,
'lightbulb',
'skyscraper',
'mountain-bike',
'fireworks',
'computer-monitor' ,
'bear',
'grand-piano', 
'kangaroo',
'laptop' ]        

train_gen = train_data_gen.flow_from_directory(
    os.path.join(dataset_dir, 'training'),
    target_size = (320, 320),
    batch_size = 8,
    class_mode = 'categorical',
    color_mode='rgb',
    classes=classes, 
    subset="training",
    shuffle=True,
    seed=SEED
)

valid_gen = train_data_gen.flow_from_directory(
    os.path.join(dataset_dir, 'test'),
    target_size = (320, 320),
    batch_size = 8,
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    subset="validation",
    shuffle=False,
    seed=SEED)

Found 1247 images belonging to 20 classes.
Found 307 images belonging to 20 classes.


In [0]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

# Transfer Learning
## Here we are using InceptionV3 implementation provided by keras

In [18]:
mn = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

87916544/87910968 [==============================] - 8s 0us/step


In [0]:
mn.summary()
for layer in mn.layers:
    print(layer, layer.trainable)


In [20]:
len(mn.layers)

311

Model Creation: 
Here tried multiple finetunings, but didn't get as much as freezing until 311 layer (so last layer) 

In [0]:
# The Model
# ------------

finetuning = True

if finetuning:
    freeze_until = 311 # layer from which we want to fine-tune
    
    for layer in mn.layers[:freeze_until]:
        layer.trainable = False
else:
    mn.trainable = False
    
model = tf.keras.Sequential()
model.add(mn)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()

## Prepare the model for training

In [0]:
# Optimization parameters
# -----------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training with early stopping

In [0]:
import os
from datetime import datetime


exps_dir = os.path.join('./', 'inceptionV3_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN_inceptionV3_no_fine_tuning'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# -------------------------------------------------------------------------------------------
# ckpt_dir = os.path.join(exp_dir, 'ckpts')
# if not os.path.exists(ckpt_dir):
#     os.makedirs(ckpt_dir)

# ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
#                                                  save_weights_only=True)  # False to save the model directly
# callbacks.append(ckpt_callback)

#--------------------------------------------------------------------------------------------

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=100,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

In [0]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform

def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (320, 320, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

dict_img_pred = {}

from tqdm import tqdm_notebook as tqdm

for i in tqdm(os.listdir(os.path.join(dataset_dir,'test'))):
  
  image = load(os.path.join(dataset_dir,'test',i))
  out_softmax = model.predict(image)
  predicted_class = tf.argmax(out_softmax, 1)
  dict_img_pred[i] = predicted_class.numpy()[0]

# finally creating submission
create_csv(dict_img_pred)